## Import

In [1]:
import os
import random

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import wandb

In [3]:
from neumeta.models import create_model_cifar10 as create_model
from neumeta.utils import (
    parse_args, print_omegaconf,
    load_checkpoint, save_checkpoint,
    set_seed,
    get_cifar10, 
    sample_coordinates, sample_subset, shuffle_coordinates_all,
    get_hypernetwork, get_optimizer, 
    sample_weights,
    weighted_regression_loss, validate_single, AverageMeter, EMA,
    sample_single_model, sample_merge_model,
)

## Functions

### 1 Find maximum dimension of the model

In [4]:
def find_max_dim(model_cls):
    """Find maximum dimension of the model"""
    # Get the learnable parameters of the model
    checkpoint = model_cls.learnable_parameter 

    # Set the maximum value to the length of the checkpoint
    max_value = len(checkpoint)

    # Iterate over the new model's weight
    for i, (k, tensor) in enumerate(checkpoint.items()):
        # Handle 2D tensors (e.g., weight matrices) 
        if len(tensor.shape) == 4:
            coords = [tensor.shape[0], tensor.shape[1]]
            max_value = max(max_value, max(coords))
        # Handle 1D tensors (e.g., biases)
        elif len(tensor.shape) == 1:
            max_value = max(max_value, tensor.shape[0])
    
    return max_value

### 2 Initialize wandb

In [5]:
def initialize_wandb(config):
    import time
    """
    Initializes Weights and Biases (wandb) with the given configuration.
    
    Args:
        configuration (dict): Configuration parameters for the run.
    """
    # Name the run using current time and configuration name
    run_name = f"{time.strftime('%Y%m%d%H%M%S')}-{config.experiment.name}"
    
    wandb.init(project="ninr-trial", name=run_name, config=dict(config), group='cifar10')

### 3 Initialize model dictionary

In [6]:
def init_model_dict(args, device):
    """
    Initializes a dictionary of models for each dimension in the given range, along with ground truth models for the starting dimension.

    Args:
        args: An object containing the arguments for initializing the models.

    Returns:
        dim_dict: A dictionary containing the models for each dimension, along with their corresponding coordinates, keys, indices, size, and ground truth models.
        gt_model_dict: A dictionary containing the ground truth models for the starting dimension.
    """
    dim_dict = {}
    gt_model_dict = {}
    
    # Create a model for each dimension in dimensions range
    for dim in args.dimensions.range:
        model_cls = create_model(args.model.type,
                                 hidden_dim=dim,
                                 path=args.model.pretrained_path,
                                 smooth=args.model.smooth).to(device)
        # Sample the coordinates, keys, indices, and the size for the model
        coords_tensor, keys_list, indices_list, size_list = sample_coordinates(model_cls)
        # Add the model, coordinates, keys, indices, size, and key mask to the dictionary
        dim_dict[f"{dim}"] = (model_cls, coords_tensor, keys_list, indices_list, size_list, None)

        # Print to makes line better
        print('\n')
        
        # If the dimension is the starting dimension (the dimension of pretrained_model), add the ground truth model to the dictionary
        if dim == args.dimensions.start:
            print(f"Loading model for dim {dim}")
            model_trained = create_model(args.model.type, 
                                         hidden_dim=dim, 
                                         path=args.model.pretrained_path, 
                                         smooth=args.model.smooth).to(device)
            model_trained.eval()
            gt_model_dict[f'{dim}'] = model_trained

    
    return dim_dict, gt_model_dict

### 4 Training function for target model of a random dimension

In [7]:
# Function to train the model for one epoch
def train_one_epoch(model, train_loader, optimizer, criterion, dim_dict, gt_model_dict, epoch_idx, ema=None, args=None, device='cpu'):
    # Set the model to training mode
    model.train()
    total_loss = 0.0

    # Initialize AverageMeter objects to track the losses
    losses = AverageMeter()
    cls_losses = AverageMeter()
    reg_losses = AverageMeter()
    reconstruct_losses = AverageMeter()

    # Iterate over the training data
    for batch_idx, (x, target) in enumerate(train_loader):
        # Zero the gradients
        optimizer.zero_grad()

        # Preprocess input
        # ------------------------------------------------------------------------------------------------------
        # Move the data to the device
        x, target = x.to(device), target.to(device)
        # Choose a random hidden dimension
        hidden_dim = random.choice(args.dimensions.range)
        # Get the model class, coordinates, keys, indices, size, and key mask for the chosen dimension
        model_cls, coords_tensor, keys_list, indices_list, size_list, key_mask = dim_dict[f"{hidden_dim}"]
        # Sample a subset the input tensor of the coordinates, keys, indices, size, and selected keys
        coords_tensor, keys_list, indices_list, size_list, selected_keys = sample_subset(coords_tensor,
                                                                                         keys_list,
                                                                                         indices_list,
                                                                                         size_list,
                                                                                         key_mask,
                                                                                         ratio=args.ratio)
        # Add noise to the coordinates if specified
        if args.training.coordinate_noise > 0.0:
            coords_tensor = coords_tensor + (torch.rand_like(coords_tensor) - 0.5) * args.training.coordinate_noise


        # Main task of hypernetwork and target network
        # ------------------------------------------------------------------------------------------------------
        # Sample the weights for the target model using hypernetwork
        model_cls, reconstructed_weights = sample_weights(model, model_cls,
                                                          coords_tensor, keys_list, indices_list, size_list, key_mask, selected_keys,
                                                          device=device, NORM=args.dimensions.norm)
        # Forward pass
        predict = model_cls(x)


        # Compute losses
        # ------------------------------------------------------------------------------------------------------
        # Compute classification loss
        cls_loss = criterion(predict, target) 
        # Compute regularization loss
        reg_loss = sum([torch.norm(w, p=2) for w in reconstructed_weights])
        # Compute reconstruction loss if ground truth model is available
        if f"{hidden_dim}" in gt_model_dict:
            gt_model = gt_model_dict[f"{hidden_dim}"]
            gt_selected_weights = [
                w for k, w in gt_model.learnable_parameter.items() if k in selected_keys]

            reconstruct_loss = weighted_regression_loss(
                reconstructed_weights, gt_selected_weights)
        else:
            reconstruct_loss = torch.tensor(0.0)
        # Compute the total loss
        loss = args.hyper_model.loss_weight.ce_weight * cls_loss + args.hyper_model.loss_weight.reg_weight * \
            reg_loss + args.hyper_model.loss_weight.recon_weight * reconstruct_loss


        # Compute gradients and update weights
        # ------------------------------------------------------------------------------------------------------
        # Zero the gradients of the updated weights
        for updated_weight in model_cls.parameters():
            updated_weight.grad = None

        # Compute the gradients of the reconstructed weights
        loss.backward(retain_graph=True)
        torch.autograd.backward(reconstructed_weights, [
                                w.grad for k, w in model_cls.named_parameters() if k in selected_keys])
        
        # Clip the gradients if specified
        if args.training.get('clip_grad', 0.0) > 0:
            torch.nn.utils.clip_grad_value_(
                model.parameters(), args.training.clip_grad)
            
        # Update the weights
        optimizer.step()

        # Update the EMA if specified
        if ema:
            ema.update()  # Update the EMA after each training step
        total_loss += loss.item()

        # Update the AverageMeter objects
        losses.update(loss.item())
        cls_losses.update(cls_loss.item())
        reg_losses.update(reg_loss.item())
        reconstruct_losses.update(reconstruct_loss.item())


        # Log (or plot) losses
        # ------------------------------------------------------------------------------------------------------
        # Log the losses and learning rate to wandb
        if batch_idx % args.experiment.log_interval == 0:
            wandb.log({
                "Loss": losses.avg,
                "Cls Loss": cls_losses.avg,
                "Reg Loss": reg_losses.avg,
                "Reconstruct Loss": reconstruct_losses.avg,
                "Learning rate": optimizer.param_groups[0]['lr']
            }, step=batch_idx + epoch_idx * len(train_loader))
            # Print the losses and learning rate
            print(
                f"Iteration {batch_idx}: Loss = {losses.avg:.4f}, Reg Loss = {reg_losses.avg:.4f}, Reconstruct Loss = {reconstruct_losses.avg:.4f}, Cls Loss = {cls_losses.avg:.4f}, Learning rate = {optimizer.param_groups[0]['lr']:.4e}")
    
    # Returns the training loss, structure of network in each dimension, and the original structure of pretrained network
    return losses.avg, dim_dict, gt_model_dict

## Main

### 0 Set device to GPU

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### 1 Parsing arguments for inputs

In [9]:
CONFIG_PATH = 'neumeta/config/cifar10/resnet20_cifar10_32-64v3_200e-coordnoise-resmlpv2_smooth_5_256_16.yaml'
RATIO = '1.0'

In [10]:
argv_train = ['--config', CONFIG_PATH, '--ratio', RATIO]

In [11]:
args = parse_args(argv_train)  # Parse arguments
print_omegaconf(args)  # Print arguments

Loading base config from toy/experiments/base_config.yaml
+--------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+
|                 Key                  |                                                                Value                                                                 |
+--------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+
|           experiment.name            |                             ninr_resnet20_cifar10_32-64-4layer-200e-noisecoord-resmlpv2_smooth_5_256_16                              |
|         experiment.recononly         |                                                                  0                                                                   |
|        experiment.num_epochs         |                      

In [12]:
set_seed(args.experiment.seed)

Setting seed... 42 for reproducibility


### 2 Get training and validation data (in dataloader format)

In [13]:
train_loader, val_loader = get_cifar10(args.training.batch_size, strong_transform=args.training.get('strong_aug', None))

### 3 Create target model

#### 3.0 Create the model

In [14]:
model = create_model(args.model.type,
                     hidden_dim=args.dimensions.start,
                     path=args.model.pretrained_path,
                     smooth=args.model.smooth).to(device)

Replace the last 2 block of layer3 with new block with hidden dim 64
Loading pretrained weights for resnet20


#### 3.1 Print the structure and shape of the model

In [15]:
model

CifarResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): Identity()
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Identity()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Identity()
    )
    (2): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Identity()
    )
  )
  (layer2): Sequential(


In [16]:
for i, (k, tensor) in enumerate(model.learnable_parameter.items()):
    print(k, tensor.shape)

layer3.2.conv1.weight torch.Size([64, 64, 3, 3])
layer3.2.conv1.bias torch.Size([64])
layer3.2.conv2.weight torch.Size([64, 64, 3, 3])
layer3.2.conv2.bias torch.Size([64])


#### 3.2 The maximum dimension of the target model

In [17]:
# Print the maximum dimension of the model
print(f'Maximum DIM: {find_max_dim(model)}')

Maximum DIM: 64


#### 3.3 Validate the accuracy of pretrained network

In [18]:
# Validate the model for the starting dimension (its pretrained form)
val_loss, acc = validate_single(model, val_loader, nn.CrossEntropyLoss(), args=args)
print(f'Initial Permutated model Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc * 100:.2f}%')

100%|██████████| 157/157 [00:04<00:00, 39.07it/s]

Initial Permutated model Validation Loss: 0.2825, Validation Accuracy: 92.60%


In [19]:
# Get the learnable parameters of the model
checkpoint = model.learnable_parameter
# Get the number of parameters
number_param = len(checkpoint)

In [20]:
# Print the keys of the parameters and the number of parameters
print(f"Parameters keys: {model.keys}")
print(f"Number of parameters to be learned: {number_param}")

Parameters keys: ['layer3.2.conv1.weight', 'layer3.2.conv1.bias', 'layer3.2.conv2.weight', 'layer3.2.conv2.bias']
Number of parameters to be learned: 4


### 4 Create the hypernetwork

#### 4.0 Create the model

In [21]:
# Get the hypermodel
hyper_model = get_hypernetwork(args, number_param)

Hyper model type: resmlp
Using scalar 0.1
num_freqs:  16 <class 'int'>


#### 4.1 Print model structure

In [22]:
hyper_model

NeRF_ResMLP_Compose(
  (positional_encoding): PositionalEncoding()
  (model): ModuleList(
    (0-3): 4 x NeRF_MLP_Residual_Scaled(
      (initial_layer): Linear(in_features=198, out_features=256, bias=True)
      (residual_blocks): ModuleList(
        (0-3): 4 x Linear(in_features=256, out_features=256, bias=True)
      )
      (scalars): ParameterList(
          (0): Parameter containing: [torch.float32 of size  (cuda:0)]
          (1): Parameter containing: [torch.float32 of size  (cuda:0)]
          (2): Parameter containing: [torch.float32 of size  (cuda:0)]
          (3): Parameter containing: [torch.float32 of size  (cuda:0)]
      )
      (act): ReLU(inplace=True)
      (output_layer): Linear(in_features=256, out_features=9, bias=True)
    )
  )
)

#### 4.2 Initialize EMA to track only a smooth version of the model weight

In [23]:
# Initialize the EMA
ema = EMA(hyper_model, decay=args.hyper_model.ema_decay)

### 5 Get Loss function, Optimizer, and Scheduler

In [24]:
criterion, val_criterion, optimizer, scheduler = get_optimizer(args, hyper_model)

In [25]:
print(f'Criterion: {criterion}\nVal_criterion: {val_criterion}\nOptimizer: {optimizer}\nScheduler: {scheduler}')

Criterion: CrossEntropyLoss()
Val_criterion: CrossEntropyLoss()
Optimizer: AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 0.001
    lr: 0.001
    maximize: False
    weight_decay: 0.01
)
Scheduler: <torch.optim.lr_scheduler.MultiStepLR object at 0x00000199546B5BD0>


### 6 Training loop

#### 6.1 Initialize training parameters

In [26]:
# Initialize the starting epoch and best accuracy
start_epoch = 0
best_acc = 0.0

#### 6.2 Directory to save the model

In [27]:
# Create the directory to save the model
os.makedirs(args.training.save_model_path, exist_ok=True)

#### 6.3 Resume training loop

In [28]:
if args.resume_from:
        print(f"Resuming from checkpoint: {args.resume_from}")
        checkpoint_info = load_checkpoint(args.resume_from, hyper_model, optimizer, ema)
        start_epoch = checkpoint_info['epoch']
        best_acc = checkpoint_info['best_acc']
        print(f"Resuming from epoch: {start_epoch}, best accuracy: {best_acc*100:.2f}%")
        # Note: If there are more elements to retrieve, do so here.

#### 6.4 Initialize wandb for plotting

In [29]:
# Initialize wandb
initialize_wandb(args)

wandb: Currently logged in as: efradosuryadi (efradosuryadi-universitas-indonesia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


#### 6.5 Initialize model dictionary for each dimension and shuffle it

In [30]:
# Initialize model dictionary
dim_dict, gt_model_dict = init_model_dict(args, device)

Replace the last 2 block of layer3 with new block with hidden dim 32
Loading pretrained weights for resnet20


Replace the last 2 block of layer3 with new block with hidden dim 33
Loading pretrained weights for resnet20


Replace the last 2 block of layer3 with new block with hidden dim 34
Loading pretrained weights for resnet20


Replace the last 2 block of layer3 with new block with hidden dim 35
Loading pretrained weights for resnet20


Replace the last 2 block of layer3 with new block with hidden dim 36
Loading pretrained weights for resnet20


Replace the last 2 block of layer3 with new block with hidden dim 37
Loading pretrained weights for resnet20


Replace the last 2 block of layer3 with new block with hidden dim 38
Loading pretrained weights for resnet20


Replace the last 2 block of layer3 with new block with hidden dim 39
Loading pretrained weights for resnet20


Replace the last 2 block of layer3 with new block with hidden dim 40
Loading pretrained weights for resnet20


R

In [31]:
gt_model_dict

{'64': CifarResNet(
   (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (bn1): Identity()
   (relu): ReLU(inplace=True)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (bn1): Identity()
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (bn2): Identity()
     )
     (1): BasicBlock(
       (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (bn1): Identity()
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (bn2): Identity()
     )
     (2): BasicBlock(
       (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (bn1): Identity()
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (bn2): Identity()
   

In [32]:
dim_dict

{'32': (CifarResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): Identity()
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): Identity()
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): Identity()
      )
      (1): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): Identity()
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): Identity()
      )
      (2): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): Identity()
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    

In [33]:
dim_dict = shuffle_coordinates_all(dim_dict)
dim_dict

{'32': (CifarResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): Identity()
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): Identity()
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): Identity()
      )
      (1): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): Identity()
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): Identity()
      )
      (2): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): Identity()
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    

#### 6.6 Hypernetwork training loop

In [34]:
args.experiment.num_epochs

200

In [35]:
# Iterate over the epochs
for epoch in range(start_epoch, args.experiment.num_epochs):
    # Train the hypernetwork to generate a model with random dimension for one epoch
    train_loss, dim_dict, gt_model_dict = train_one_epoch(hyper_model, train_loader, optimizer, criterion, 
                                                          dim_dict, gt_model_dict, epoch_idx=epoch, ema=ema, 
                                                          args=args, device=device)
    # Step the scheduler
    scheduler.step()

    # Print the training loss and learning rate
    print(f"Epoch [{epoch+1}/{args.experiment.num_epochs}], Training Loss: {train_loss:.4f}, Learning Rate: {scheduler.get_last_lr()[0]:.6f}")

    # If it's time to evaluate the model
    if (epoch + 1) % args.experiment.eval_interval == 0:
        # Apply EMA if it is specified
        if ema:
            ema.apply()  # Save the weights of original model created before training_loop
        
        # Sample the merged model (create model of same structure before training loop by using the hypernetwork)
        # And then test the performance of the hypernetwork by seeing how good it is in generating the weights
        model = sample_merge_model(hyper_model, model, args) 
        # Validate the merged model
        val_loss, acc = validate_single(model, val_loader, val_criterion, args=args)

        # If EMA is specified, restore the original weights
        if ema:
            ema.restore()  # Restore the original weights to the weights of the pretrained networks

        # Log the validation loss and accuracy to wandb
        wandb.log({
            "Validation Loss": val_loss,
            "Validation Accuracy": acc
        })
        # Print the validation loss and accuracy
        print(f"Epoch [{epoch+1}/{args.experiment.num_epochs}], Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc*100:.2f}%")
        print('\n\n')

        # Save the checkpoint if the accuracy is better than the previous best
        if acc > best_acc:
            best_acc = acc
            save_checkpoint(f"{args.training.save_model_path}/cifar10_nerf_best.pth",hyper_model,optimizer,ema,epoch,best_acc)
            print(f"Checkpoint saved at epoch {epoch} with accuracy: {best_acc*100:.2f}%")


Iteration 0: Loss = 1.9313, Reg Loss = 3.5250, Reconstruct Loss = 0.0000, Cls Loss = 1.9310, Learning rate = 1.0000e-03
Iteration 100: Loss = 1.9304, Reg Loss = 6.4627, Reconstruct Loss = 0.0008, Cls Loss = 1.9290, Learning rate = 1.0000e-03
Iteration 200: Loss = 1.8972, Reg Loss = 5.9850, Reconstruct Loss = 0.0012, Cls Loss = 1.8954, Learning rate = 1.0000e-03
Iteration 300: Loss = 1.8830, Reg Loss = 6.1283, Reconstruct Loss = 0.0013, Cls Loss = 1.8811, Learning rate = 1.0000e-03
Iteration 400: Loss = 1.8735, Reg Loss = 6.5342, Reconstruct Loss = 0.0013, Cls Loss = 1.8716, Learning rate = 1.0000e-03
Iteration 500: Loss = 1.8686, Reg Loss = 6.7784, Reconstruct Loss = 0.0012, Cls Loss = 1.8667, Learning rate = 1.0000e-03
Iteration 600: Loss = 1.8639, Reg Loss = 6.8017, Reconstruct Loss = 0.0011, Cls Loss = 1.8621, Learning rate = 1.0000e-03
Iteration 700: Loss = 1.8618, Reg Loss = 7.2133, Reconstruct Loss = 0.0009, Cls Loss = 1.8602, Learning rate = 1.0000e-03
Epoch [1/200], Training Lo

100%|██████████| 157/157 [00:02<00:00, 76.52it/s]


Epoch [1/200], Validation Loss: 1.8112, Validation Accuracy: 72.10%



Checkpoint saved at epoch 0 with accuracy: 72.10%
Iteration 0: Loss = 1.7838, Reg Loss = 9.1052, Reconstruct Loss = 0.0000, Cls Loss = 1.7829, Learning rate = 1.0000e-03
Iteration 100: Loss = 1.8466, Reg Loss = 7.3239, Reconstruct Loss = 0.0013, Cls Loss = 1.8445, Learning rate = 1.0000e-03
Iteration 200: Loss = 1.8465, Reg Loss = 6.4827, Reconstruct Loss = 0.0009, Cls Loss = 1.8450, Learning rate = 1.0000e-03
Iteration 300: Loss = 1.8455, Reg Loss = 6.0528, Reconstruct Loss = 0.0008, Cls Loss = 1.8441, Learning rate = 1.0000e-03
Iteration 400: Loss = 1.8456, Reg Loss = 6.2221, Reconstruct Loss = 0.0009, Cls Loss = 1.8441, Learning rate = 1.0000e-03
Iteration 500: Loss = 1.8442, Reg Loss = 6.6371, Reconstruct Loss = 0.0010, Cls Loss = 1.8425, Learning rate = 1.0000e-03
Iteration 600: Loss = 1.8431, Reg Loss = 7.2685, Reconstruct Loss = 0.0013, Cls Loss = 1.8411, Learning rate = 1.0000e-03
Iteration 700: Loss = 1.840

100%|██████████| 157/157 [00:03<00:00, 42.04it/s]


Epoch [2/200], Validation Loss: 1.8076, Validation Accuracy: 73.30%



Checkpoint saved at epoch 1 with accuracy: 73.30%
Iteration 0: Loss = 1.8247, Reg Loss = 22.2345, Reconstruct Loss = 0.0000, Cls Loss = 1.8225, Learning rate = 1.0000e-03
Iteration 100: Loss = 1.8435, Reg Loss = 21.4041, Reconstruct Loss = 0.0014, Cls Loss = 1.8400, Learning rate = 1.0000e-03
Iteration 200: Loss = 1.8460, Reg Loss = 19.0769, Reconstruct Loss = 0.0019, Cls Loss = 1.8422, Learning rate = 1.0000e-03
Iteration 300: Loss = 1.8448, Reg Loss = 15.7357, Reconstruct Loss = 0.0016, Cls Loss = 1.8416, Learning rate = 1.0000e-03
Iteration 400: Loss = 1.8446, Reg Loss = 15.3516, Reconstruct Loss = 0.0016, Cls Loss = 1.8415, Learning rate = 1.0000e-03
Iteration 500: Loss = 1.8448, Reg Loss = 14.9491, Reconstruct Loss = 0.0014, Cls Loss = 1.8419, Learning rate = 1.0000e-03
Iteration 600: Loss = 1.8419, Reg Loss = 14.8255, Reconstruct Loss = 0.0014, Cls Loss = 1.8390, Learning rate = 1.0000e-03
Iteration 700: Loss 

...

Epoch [198/200], Validation Loss: 0.3949, Validation Accuracy: 91.54%



Iteration 0: Loss = 0.0466, Reg Loss = 65.8899, Reconstruct Loss = 0.0271, Cls Loss = 0.0129, Learning rate = 1.0000e-05
Iteration 100: Loss = 0.0508, Reg Loss = 64.1903, Reconstruct Loss = 0.0005, Cls Loss = 0.0439, Learning rate = 1.0000e-05
Iteration 200: Loss = 0.0459, Reg Loss = 64.2664, Reconstruct Loss = 0.0009, Cls Loss = 0.0386, Learning rate = 1.0000e-05
Iteration 300: Loss = 0.0444, Reg Loss = 64.3702, Reconstruct Loss = 0.0010, Cls Loss = 0.0369, Learning rate = 1.0000e-05
Iteration 400: Loss = 0.0442, Reg Loss = 64.4082, Reconstruct Loss = 0.0009, Cls Loss = 0.0368, Learning rate = 1.0000e-05
Iteration 500: Loss = 0.0428, Reg Loss = 64.3859, Reconstruct Loss = 0.0009, Cls Loss = 0.0355, Learning rate = 1.0000e-05
Iteration 600: Loss = 0.0435, Reg Loss = 64.3869, Reconstruct Loss = 0.0008, Cls Loss = 0.0363, Learning rate = 1.0000e-05
Iteration 700: Loss = 0.0439, Reg Loss = 64.3812, Reconstruct Loss =

100%|██████████| 157/157 [00:02<00:00, 73.85it/s]


Epoch [199/200], Validation Loss: 0.4051, Validation Accuracy: 91.32%



Iteration 0: Loss = 0.0734, Reg Loss = 65.9154, Reconstruct Loss = 0.0000, Cls Loss = 0.0668, Learning rate = 1.0000e-05
Iteration 100: Loss = 0.0416, Reg Loss = 64.5216, Reconstruct Loss = 0.0005, Cls Loss = 0.0346, Learning rate = 1.0000e-05
Iteration 200: Loss = 0.0453, Reg Loss = 64.5531, Reconstruct Loss = 0.0010, Cls Loss = 0.0378, Learning rate = 1.0000e-05
Iteration 300: Loss = 0.0457, Reg Loss = 64.5367, Reconstruct Loss = 0.0007, Cls Loss = 0.0385, Learning rate = 1.0000e-05
Iteration 400: Loss = 0.0454, Reg Loss = 64.5234, Reconstruct Loss = 0.0006, Cls Loss = 0.0383, Learning rate = 1.0000e-05
Iteration 500: Loss = 0.0453, Reg Loss = 64.5019, Reconstruct Loss = 0.0006, Cls Loss = 0.0383, Learning rate = 1.0000e-05
Iteration 600: Loss = 0.0441, Reg Loss = 64.4987, Reconstruct Loss = 0.0006, Cls Loss = 0.0370, Learning rate = 1.0000e-05
Iteration 700: Loss = 0.0438, Reg Loss = 64.4882, Reconstruct Loss =

100%|██████████| 157/157 [00:02<00:00, 73.29it/s]

Epoch [200/200], Validation Loss: 0.4004, Validation Accuracy: 91.34%





In [36]:
# Validate the model for the starting dimension (its pretrained form)
val_loss, acc = validate_single(dim_dict['32'][0], val_loader, nn.CrossEntropyLoss(), args=args)
print(f'Initial Permutated model Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc * 100:.2f}%')

100%|██████████| 157/157 [00:02<00:00, 76.64it/s]

Initial Permutated model Validation Loss: 0.4375, Validation Accuracy: 90.89%


In [37]:
# Validate the model for the starting dimension (its pretrained form)
val_loss, acc = validate_single(dim_dict['64'][0], val_loader, nn.CrossEntropyLoss(), args=args)
print(f'Initial Permutated model Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc * 100:.2f}%')

100%|██████████| 157/157 [00:02<00:00, 76.26it/s]

Initial Permutated model Validation Loss: 0.4127, Validation Accuracy: 91.46%


In [38]:
# Validate the model for the starting dimension (its pretrained form)
val_loss, acc = validate_single(gt_model_dict['64'], val_loader, nn.CrossEntropyLoss(), args=args)
print(f'Initial Permutated model Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc * 100:.2f}%')

100%|██████████| 157/157 [00:02<00:00, 76.50it/s]

Initial Permutated model Validation Loss: 0.2825, Validation Accuracy: 92.60%


In [39]:
# End the wandb tracking
wandb.finish()

Cls Loss,███▇▆▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Learning rate,███████████████████▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
Loss,██▇▅▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Reconstruct Loss,█▁▇▆▇▃▇▁▆▅▅▅▄▄▄▄▁▄▃▄▁▄▁▄▃▁▄▄▄▃▃▁▂▆▄▄▁▃▄▄
Reg Loss,▁███▆▆▄▅▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Validation Accuracy,▅▂▁▆▇▇██████████████████████████████████
Validation Loss,█▆▅▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Cls Loss,0.03671
Learning rate,1e-05
Loss,0.04377
Reconstruct Loss,0.00061


### 7 Testing loop

In [40]:
saved_hypernet_path = args.training.save_model_path + '/cifar10_nerf_best.pth'

In [41]:
hyper_model_test = get_hypernetwork(args, number_param)

Hyper model type: resmlp
Using scalar 0.1
num_freqs:  16 <class 'int'>


In [42]:
checkpoint = torch.load(saved_hypernet_path, map_location="cpu")  # or "cuda" if using GPU
hyper_model_test.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [43]:
for hidden_dim in range(16, 65):
    # Create a model for this given dimension
    model = create_model(args.model.type,
                         hidden_dim=hidden_dim,
                         path=args.model.pretrained_path,
                         smooth=args.model.smooth).to(device)
    
    # If EMA is specified, apply it
    if ema:
        print('Applying EMA')
        ema.apply()

    # Sample the merged model
    accumulated_model = sample_merge_model(hyper_model_test, model, args, K=100)

    # Validate the merged model
    val_loss, acc = validate_single(accumulated_model, val_loader, val_criterion, args=args)

    # If EMA is specified, restore the original weights after applying EMA
    if ema:
        ema.restore()  # Restore the original weights after applying 
        
    # Save the model
    save_name = os.path.join(args.training.save_model_path, f"cifar10_{accumulated_model.__class__.__name__}_dim{hidden_dim}_single.pth")
    torch.save(accumulated_model.state_dict(),save_name)

    # Print the results
    print(f"Test using model {args.model}: hidden_dim {hidden_dim}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc*100:.2f}%")
    print('\n')

    # Define the directory and filename structure
    filename = f"cifar10_results_{args.experiment.name}.txt"
    filepath = os.path.join(args.training.save_model_path, filename)

    # Write the results. 'a' is used to append the results; a new file will be created if it doesn't exist.
    with open(filepath, "a") as file:
        file.write(f"Hidden_dim: {hidden_dim}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc*100:.2f}%\n")


Replace the last 2 block of layer3 with new block with hidden dim 16
Loading pretrained weights for resnet20
Applying EMA


100%|██████████| 157/157 [00:02<00:00, 72.28it/s]


Test using model {'type': 'ResNet20', 'pretrained_path': 'toy/checkpoint/cifar10_ResNet20_smooth_Ours.pth', 'smooth': False}: hidden_dim 16, Validation Loss: 0.5090, Validation Accuracy: 88.24%


Replace the last 2 block of layer3 with new block with hidden dim 17
Loading pretrained weights for resnet20
Applying EMA


100%|██████████| 157/157 [00:02<00:00, 71.34it/s]


Test using model {'type': 'ResNet20', 'pretrained_path': 'toy/checkpoint/cifar10_ResNet20_smooth_Ours.pth', 'smooth': False}: hidden_dim 17, Validation Loss: 0.6210, Validation Accuracy: 85.78%


Replace the last 2 block of layer3 with new block with hidden dim 18
Loading pretrained weights for resnet20
Applying EMA


...

Test using model {'type': 'ResNet20', 'pretrained_path': 'toy/checkpoint/cifar10_ResNet20_smooth_Ours.pth', 'smooth': False}: hidden_dim 62, Validation Loss: 0.4183, Validation Accuracy: 91.32%


Replace the last 2 block of layer3 with new block with hidden dim 63
Loading pretrained weights for resnet20
Applying EMA


100%|██████████| 157/157 [00:02<00:00, 73.10it/s]


Test using model {'type': 'ResNet20', 'pretrained_path': 'toy/checkpoint/cifar10_ResNet20_smooth_Ours.pth', 'smooth': False}: hidden_dim 63, Validation Loss: 0.4086, Validation Accuracy: 91.48%


Replace the last 2 block of layer3 with new block with hidden dim 64
Loading pretrained weights for resnet20
Applying EMA


100%|██████████| 157/157 [00:02<00:00, 73.09it/s]

Test using model {'type': 'ResNet20', 'pretrained_path': 'toy/checkpoint/cifar10_ResNet20_smooth_Ours.pth', 'smooth': False}: hidden_dim 64, Validation Loss: 0.4185, Validation Accuracy: 91.00%


